# Library Imports and Data Preprocessing

In [ ]:
!pip install git+https://github.com/huggingface/transformers
!pip install datasets

In [ ]:
from datasets import load_dataset, Audio
from transformers import AutoFeatureExtractor

In [ ]:
gtzan = load_dataset("marsyas/gtzan", "all")

In [ ]:
gtzan = gtzan["train"].train_test_split(seed=42, shuffle=True, test_size=0.1)
gtzan

In [ ]:
id2label_fn = gtzan["train"].features["genre"].int2str
id2label_fn(gtzan["train"][0]["genre"])

In [ ]:
model_id = "duysal/distilhubert-finetuned-gtzan-5-epochs-finetuned-gtzan"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)

In [ ]:
sampling_rate = feature_extractor.sampling_rate
sampling_rate

In [ ]:
from datasets import Audio

gtzan = gtzan.cast_column("audio", Audio(sampling_rate=sampling_rate))
gtzan["train"][0]

In [ ]:
import numpy as np

sample = gtzan["train"][0]["audio"]

print(f"Mean: {np.mean(sample['array']):.3}, Variance: {np.var(sample['array']):.3}")

In [ ]:
inputs = feature_extractor(sample["array"], sampling_rate=sample["sampling_rate"])

print(f"inputs keys: {list(inputs.keys())}")

print(
    f"Mean: {np.mean(inputs['input_values']):.3}, Variance: {np.var(inputs['input_values']):.3}"
)

In [ ]:
max_duration = 30.0


def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
        return_attention_mask=True,
    )
    return inputs

In [ ]:
gtzan_encoded = gtzan.map(
    preprocess_function,
    remove_columns=["audio", "file"],
    batched=True,
    batch_size=100,
    num_proc=1,
)
gtzan_encoded

In [ ]:
gtzan_encoded = gtzan_encoded.rename_column("genre", "label")

In [ ]:
id2label = {
    str(i): id2label_fn(i)
    for i in range(len(gtzan_encoded["train"].features["label"].names))
}
label2id = {v: k for k, v in id2label.items()}

id2label["7"]

In [ ]:
from transformers import AutoModelForAudioClassification, AutoConfig

num_labels = len(id2label)

config = AutoConfig.from_pretrained(model_id)
config.hidden_dropout_prob = 0.3  # Increase from default 0.1
config.attention_probs_dropout_prob = 0.3
config.num_labels = num_labels
config.label2id = label2id
config.id2label = id2label

model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    config=config
)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

# Fine-Tuning the Model

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from transformers import TrainingArguments

model_name = model_id.split("/")[-1]
batch_size = 2
gradient_accumulation_steps = 4
num_train_epochs = 3

training_args = TrainingArguments(
    f"{model_name}-finetuned-gtzan",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    push_to_hub=True,
    weight_decay=0.05,
)

In [ ]:
!pip install evaluate

In [ ]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=gtzan_encoded["train"],
    eval_dataset=gtzan_encoded["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
kwargs = {
    "dataset_tags": "marsyas/gtzan",
    "dataset": "GTZAN",
    "model_name": f"{model_name}-finetuned-gtzan",
    "finetuned_from": model_id,
    "tasks": "audio-classification",
}

In [ ]:
trainer.push_to_hub(**kwargs)